In [ ]:
from dolfin import *
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Mesh data
mesh_path = '/mnt/d/Meshes/'
mesh_name = 'Chip_Micronit_14e5Elem'

# Load mesh from xml file ~ mesh created with gmsh
mesh_xml = Mesh(mesh_path + mesh_name + '.xml')

# Print mesh dimension
tdim = mesh_xml.topology().dim()
print('Mesh dimension: ', tdim)

# Print number of vertexes in mesh
nvert = len(mesh_xml.coordinates())
print("Number of vertexes: ", nvert)

In [ ]:
# Load boundaries and subdomains
n = FacetNormal(mesh_xml)
boundaries = MeshFunction('size_t', mesh_xml,mesh_path + mesh_name + '_facet_region.xml')
markers = MeshFunction('size_t', mesh_xml,mesh_path + mesh_name + '_physical_region.xml')
ds = Measure('ds', domain=mesh_xml, subdomain_data=boundaries)

# Get boundary marks from mesh (created with gmsh)
obstacleTags = [1]
inletTag = 2
outletTag = 3
Wall1Tag = 4
Wall2Tag = 5
fluidTag = 6

In [ ]:
# Pressure Difference
# UInlet = 0.5

# Fluid Properties
rho = 1
mu = 1
# nu = Constant(0.01)

# Boundary conditions for inlet and outlet
pInlet = 15000.0    # Pa
pOutlet = 0.0   # Pa

# Correction factor considering physical etching of the microfluidic device
corr_factor = 2e-5 #m ~ chip thickness correction factor/ Quasi-2D state

# Mesh Elements
# Velocity
velocityElementfamily = 'Lagrange'
velocityElementOrder = 2
# Pressure
pressureElementfamily = 'Lagrange'
pressureElementOrder = 1

In [ ]:
# Get Element Shape: Triangle, etc...
elementShape = mesh_xml.ufl_cell()

# Set Mesh Elements
Uel = VectorElement(velocityElementfamily, elementShape, velocityElementOrder) # Velocity vector field
Pel = FiniteElement(pressureElementfamily, elementShape, pressureElementOrder) # Pressure field
UPel = MixedElement([Uel,Pel])

In [ ]:
# Function Spaces: Velocity and Pressure
U = VectorFunctionSpace(mesh_xml, velocityElementfamily, velocityElementOrder)
P = FunctionSpace(mesh_xml, pressureElementfamily, pressureElementOrder)

# Mixed Function Space: Pressure and Velocity
W = FunctionSpace(mesh_xml,UPel)

In [ ]:
# Define test functions
(v,q) = TestFunctions(W)

# Define trial functions
w = Function(W)
(u,p) = (as_vector((w[0], w[1])), w[2])

In [ ]:
# Apply boundary conditions
bc = []
# No-slip condition for walls

for i in obstacleTags:
    bc0 = DirichletBC(W.sub(0), Constant((0.0,0.0)), boundaries, i)
    bc.append(bc0)

bc.append(DirichletBC(W.sub(0), Constant((0.0,0.0)), boundaries, Wall1Tag))
bc.append(DirichletBC(W.sub(0), Constant((0.0,0.0)), boundaries, Wall2Tag))

# Inlet and outlet pressure conditions
bc.append(DirichletBC(W.sub(1), Constant(pInlet), boundaries, inletTag))
bc.append(DirichletBC(W.sub(1), Constant(pOutlet), boundaries, outletTag))

# bc = [bc0, bc1, bc2, PInlet, POutlet]

In [ ]:
# Variational form

# Linear Momentum Equation
    # Inertia Term            # Viscous Term                  # Pressure Term           # Continuity
F = inner(grad(u)*u, v)*dx() + nu*inner(grad(u), grad(v))*dx() - div(v)*p*dx() + q*div(u)*dx()

dw = TrialFunction(W)

In [ ]:
# Calculate Jacobian Matrix
J = derivative(F,w,dw)

In [ ]:
# Problem and Solver definitions
nsproblem = NonlinearVariationalProblem(F, w, bc, J)
solver = NonlinearVariationalSolver(nsproblem)

In [ ]:
# Solve problem
solver.solve()
u,p=split(w)

In [ ]:
# Crop dataset in a N x M grid and bundle the patches in a [mesh, vel, pressure] file
# (If dataset is converted into numpy array format, save using Pickle)
# Save dataset
from dl_utils import save_data
save_data(mesh_xml, t, u, p, path)